In [1]:
import pandas as pd

df_copy = pd.read_excel('f8c69a2b-71b1-40b0-b70b-0d0b35bfd921.xlsx')

In [2]:
df = df_copy.copy()

# Para testes com quantidade de dias menor que 1 ano
df.at[df.index[0], 'Dias restantes para prescrição'] = 1

In [3]:
df_estado_vara = df.copy()

estado = 'CE'
vara = '27ª Vara Federal'
tabela_query_exemplo = df_estado_vara[(df_estado_vara['Localização_Prescrição'] ==  estado) & (df_estado_vara['Serventia Descrição Prescrição'] == vara)]

dict_new_nomes = {'Localização_Prescrição': 'Estado', 'Serventia Descrição Prescrição': 'Vara Federal', 'Dias restantes para prescrição':'Dias restantes'}

colunas = ['Número do processo', 'Localização_Prescrição', 'Serventia Descrição Prescrição', 'Dias restantes para prescrição']

df_menos_1_ano = tabela_query_exemplo[tabela_query_exemplo['Dias restantes para prescrição'] < 365][colunas].reset_index(drop=True).rename(columns=dict_new_nomes)
df_entre_1_e_2 = tabela_query_exemplo[(tabela_query_exemplo['Dias restantes para prescrição'] >= 365) & (tabela_query_exemplo['Dias restantes para prescrição'] < 365*2)][colunas].reset_index(drop=True).rename(columns=dict_new_nomes)
df_entre_2_e_3 = tabela_query_exemplo[(tabela_query_exemplo['Dias restantes para prescrição'] >= 365*2) & (tabela_query_exemplo['Dias restantes para prescrição'] < 365*3)][colunas].reset_index(drop=True).rename(columns=dict_new_nomes)

# Start indice 1
df_menos_1_ano.index += 1
df_entre_1_e_2.index += 1
df_entre_2_e_3.index += 1

In [4]:
import numpy as np
condicoes = [
    (df['Dias restantes para prescrição'] >= 365*3),
    (df['Dias restantes para prescrição'] >= 365*2) & (df['Dias restantes para prescrição'] < 365*3),
    (df['Dias restantes para prescrição'] >= 365) & (df['Dias restantes para prescrição'] < 365*2),
    (df['Dias restantes para prescrição'] < 365)
]

resultados = [
    'Processos acima de 3 anos para prescrição',
    'Processos entre 2 anos e 3 anos para prescrição',
    'Processos entre 1 ano e 2 anos para prescrição',
    'Processos com menos de 1 ano para prescrição'
]

df['Dias Grupos'] = np.select(condicoes, resultados)

df['Dias Grupos'].value_counts()

Processos entre 1 ano e 2 anos para prescrição     1746
Processos acima de 3 anos para prescrição            85
Processos entre 2 anos e 3 anos para prescrição      61
Processos com menos de 1 ano para prescrição          1
Name: Dias Grupos, dtype: int64

In [5]:
df['Dias Grupos'].head()

tabela_nova = df.groupby(['Localização_Prescrição', 'Serventia Descrição Prescrição', 'Dias Grupos'])['Número do processo'].nunique()

tabela_redalert =pd.DataFrame(tabela_nova)

In [6]:
tabela_redalert.reset_index(inplace=True)

#tabela_redalert = tabela_redalert.rename(columns={'Localização': 'Seção Judiciária', 'Dias Grupos':'Alerta', '%ID_PROCESSO_TRF':'Quantidade'})

#tabela_redalert.set_index(['Seção Judiciária', 'Alerta']).unstack(level=1)['Quantidade']

tabela_redalert = tabela_redalert.pivot_table(index=['Localização_Prescrição', 'Serventia Descrição Prescrição'], columns=['Dias Grupos'], values='Número do processo', aggfunc='sum').reset_index()


#tabela_nova.pivot_table('Seção Judiciária', 'Alerta', 'Quantidade', aggfunc='first')
tabela_redalert


Dias Grupos,Localização_Prescrição,Serventia Descrição Prescrição,Processos acima de 3 anos para prescrição,Processos com menos de 1 ano para prescrição,Processos entre 1 ano e 2 anos para prescrição,Processos entre 2 anos e 3 anos para prescrição
0,AL,11ª Vara Federal,2.0,NaN,43.0,NaN
1,AL,12ª Vara Federal,NaN,NaN,4.0,NaN
2,AL,13ª Vara Federal,NaN,NaN,10.0,NaN
3,AL,1ª Vara Federal,1.0,NaN,7.0,NaN
4,AL,2ª Vara Federal,2.0,NaN,13.0,NaN
...,...,...,...,...,...,...
74,SE,3ª Vara Federal,4.0,NaN,12.0,1.0
75,SE,6ª Vara Federal,1.0,NaN,34.0,NaN
76,SE,7ª Vara Federal,1.0,NaN,9.0,NaN
77,SE,8ª Vara Federal,NaN,NaN,14.0,1.0


In [7]:
#tabela_redalert.drop('Alerta', axis=0, inplace=True)
colunas_numericas = tabela_redalert.select_dtypes(include=['float64', 'int64']).columns
tabela_redalert[colunas_numericas] = tabela_redalert[colunas_numericas].fillna(0).astype(int)

tabela_redalert


Dias Grupos,Localização_Prescrição,Serventia Descrição Prescrição,Processos acima de 3 anos para prescrição,Processos com menos de 1 ano para prescrição,Processos entre 1 ano e 2 anos para prescrição,Processos entre 2 anos e 3 anos para prescrição
0,AL,11ª Vara Federal,2,0,43,0
1,AL,12ª Vara Federal,0,0,4,0
2,AL,13ª Vara Federal,0,0,10,0
3,AL,1ª Vara Federal,1,0,7,0
4,AL,2ª Vara Federal,2,0,13,0
...,...,...,...,...,...,...
74,SE,3ª Vara Federal,4,0,12,1
75,SE,6ª Vara Federal,1,0,34,0
76,SE,7ª Vara Federal,1,0,9,0
77,SE,8ª Vara Federal,0,0,14,1


In [8]:
datatoexcel = pd.ExcelWriter('testes_prescricao.xlsx')

tabela_redalert.to_excel(datatoexcel)

datatoexcel.save()

In [9]:

tabela_redalert = tabela_redalert[['Localização_Prescrição', 'Serventia Descrição Prescrição',
       'Processos com menos de 1 ano para prescrição',
       'Processos entre 1 ano e 2 anos para prescrição',
       'Processos entre 2 anos e 3 anos para prescrição']]

tabela_redalert = tabela_redalert.reset_index(drop=True)

In [10]:
from datetime import datetime

# Obtendo a data de hoje como um objeto datetime
data_hoje = datetime.now()

# Convertendo a data de hoje para uma string no formato 'YYYY-MM-DD'
data_hoje_str = data_hoje.strftime('%Y-%m-%d')

In [3]:
import pdfkit
from sender import Sender

envio = Sender()

body = """\
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Relatório Mensal de Prescrição e Improbidade Administrativa</title>
    <style>
        body {
            font-family: "Verdana", "sans-serif";
        }

        .titulo {
            font-size: 1.5em;
            font-weight: bold;
            text-align: center;
            background-color: #ADADAD;
            padding: 0.5em 0 0.5em 0.5em;
            position: relative;
            margin-bottom: 0;
        }

        .subtitulo {
            font-size: 0.8em;
            text-align: center;
            background-color: #E8E8E8;
            padding: 0.5em 0 0.5em 0.5em;
            margin-top: 0;
            position: relative;
            top: 0;
        }

        .secao {
            font-size: 1.2em;
            text-align: center;
            font-weight: bold;
            background-color: #E8E8E8;
            padding: 0.5em 0 0.5em 0.5em;
            top: 0;
        }

        .lista {
            margin: 0.1em 0 0.2em 1em;
        }

        li {
            margin-bottom: 0.4em;
        }

        .tabela_de_imagens {
            align-items: center;
            margin-top: 2em;
            margin-bottom: 2em;
        }

        .footer {
            text-align: center;
            background-color: #ADADAD;
            width: 100%;
            padding: 1em 0;
            margin-top: 2em;
        }

        .logo_residencia {
            width: 14em;
            padding: 0 2em;
        }

        .logo_ra {
            width: 3.5em;
        }

        .logo_trf5 {
            width: 5em;
        }

        .nota_email {
            font-size: 0.8em;
            color: #6b6a6a;
            margin-top: 0.5em;
        }

        table {
            margin: auto;
        }

        td {
            padding-top: 5px;
            text-align: center;
            font-weight: none;
            
        }

        tr{
            padding-top: 5px;
            font-weight: bold;
            background-color: #E8E8E8;
        }

        tr:hover {
            background-color: #6b6a6a;
            color: #fff;
        }
    </style>
</head>


<body>

<img src="pie_chart.png"></img>

    <div>
        <p class="titulo">Relatório Mensal Prescrição em Improbidade
            Administrativa e Geral do Estado """+estado+""" <br>"""+vara+"""
            <span id="sigla"></span>
        </p>

        <p class="subtitulo">Data de extração: <strong id="data_extracao">"""+data_hoje_str+"""</strong></p>
    </div>

    <div>
        <div>
            <p class="secao">
                Processos com menos de 1 ano
            </p>
            """+df_menos_1_ano.to_html()+"""   
        </div>

        <div>
            <p class="secao">
                Processos entre 1 ano e 2 anos para prescrição
            </p>
            """+df_entre_1_e_2.to_html()+"""

        </div>
        <div>
            <p class="secao">
                Processos entre 2 anos e 3 anos para prescrição
            </p>
            """+df_entre_2_e_3.to_html()+"""
        </div>
    </div>

    <div class="footer">
        <strong>Para mais informações acesse o Instant:</strong><br>
        <a href="https://instant.trf5.jus.br/inspecao-web/#/login">
            https://instant.trf5.jus.br/inspecao-web/#/login
        </a>
    </div>
</body>

</html>
"""

from xhtml2pdf import pisa

def convert_html_to_pdf(html_string, pdf_path):
    with open(pdf_path, "wb") as pdf_file:
        pisa_status = pisa.CreatePDF(html_string, dest=pdf_file)
        
    return not pisa_status.err

# Generate PDF
pdf_path = "relatorio.pdf"
if convert_html_to_pdf(body, pdf_path):
    print(f"PDF generated and saved at {pdf_path}")
else:
    print("PDF generation failed")

envio.enviar_email(body)

NameError: name 'estado' is not defined